In [ ]:
import numpy as np

array = np.array([
    [2, 3, 5, 2, 3, 5, 2, 3],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [1, 1, 4, 1, 1, 4, 1, 1],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [1, 1, 4, 1, 1, 4, 1, 1],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [3, 2, 5, 3, 2, 5, 3, 2]
])

start = (3,1)

def calculate_distance(pos1, pos2):
    return np.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

def find_nearest_exit(map_array, start):
    pos_of_exits = np.argwhere(map_array == 3)
    pos_of_exits.tolist()
    distances = [calculate_distance(start, (x, y)) for x, y in pos_of_exits]
    min_distance = min(distances)
    closest_position = pos_of_exits[distances.index(min_distance)]
    
    return (closest_position[0],closest_position[1])

In [ ]:
import heapq
import numpy as np

def heuristic(a, b, array,raw_orientation=None,new_orientation=None):
    """
    计算两个点之间的曼哈顿距离
    """
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def calculate_distance(pos1, pos2):
    return np.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

#搜索map中最近的出口
def find_nearest_exit(map_array, start):
    pos_of_exits = np.argwhere(map_array == 3)
    pos_of_exits.tolist()
    distances = [calculate_distance(start, (x, y)) for x, y in pos_of_exits]
    min_distance = min(distances)
    closest_position = pos_of_exits[distances.index(min_distance)]
    
    return (closest_position[0],closest_position[1])

def find_neighbors(point,array=None):
    neighbors=[]
    if point[0] in [1,4,7]:
        neighbors.append((0,1))
    if point[0] in [2,5,8]:
        neighbors.append((0,-1))
    if point[1] in [0,3,6]:
        neighbors.append((1,0))
    if point[1] in [1,4,7]:
        neighbors.append((-1,0))
    return neighbors



def Astar(array, start, goal):
    """
    A* 寻路算法
    """

    close_set = set()
    came_from = {}
    gscore = {start:0}
    fscore = {start:heuristic(start, goal, array)}
    oheap = []

    #初始化方向，位于上方入口（行索引为0），初始方向向下；位于下方入口，初始方向向上
    if start[0] == 0:
        last_orientation = (1,0)
    if start[0] == len(array)-1:
        last_orientation = (-1,0)
    if start[0] in [2,5]:
        last_orientation = (0,-1)
    if start[0] in [4,7]:
        last_orientation = (0,1)
    if start[1] in [1,4]:
        last_orientation = (-1,0)
    if start[1] in [3,6]:
        last_orientation = (1,0)
    
    heapq.heappush(oheap, (fscore[start], start))

    while oheap:
        # 标记一下这次有没有找到路
        current = heapq.heappop(oheap)[1]
        close_set.add(current)
        # 判断是否成功到达卸货单元
        for neighbor in [(0,1),(0,-1),(1,0),(-1,0)]:
            if (current[0]+neighbor[0],current[1]+neighbor[1]) == goal:
                data = []
                while current in came_from:
                    data.append(current)
                    current = came_from[current]
                data.append(start)
                return data[::-1]
        # 判断该点可达的邻居
        neighbors=find_neighbors(current,array)
        for i, j in neighbors:
            # 判断是否转向
            if current in came_from.keys():
               if current[0] != came_from[current][0] or current[1] != came_from[current][1]:
                   last_orientation=(current[0]-came_from[current][0],current[1]-came_from[current][1]) #若停留的话last_orientation会成为(0,0),修改后停留时方向不变
                    
            neighbor = current[0] + i, current[1] + j

            # 约束：1.静态障碍物; 2.地图边界； 3.动态障碍
            if 0 <= neighbor[0] < array.shape[0]:
                if 0 <= neighbor[1] < array.shape[1]:
                    # 判断是否为障碍物 此处可以改成其他限制条件
                    if array[neighbor[0]][neighbor[1]] == 5:
                        continue
                    if array[neighbor[0]][neighbor[1]] == 4 and neighbor != goal:
                        continue
                else:
                    # 超出边界
                    continue
            else:
                # 超出边界
                continue
            
            # 如果要转向，距离额外加1
            if last_orientation!=(i,j):
                tentative_g_score = gscore[current] + heuristic(current, neighbor, array) + 1  
            else:
                tentative_g_score = gscore[current] + heuristic(current, neighbor, array)
            # 如果距离更远，排除
            if neighbor in close_set  and tentative_g_score > gscore.get(neighbor, 0):
                continue
            # 如果距离更近，更新
            if  tentative_g_score <= gscore.get(neighbor, 0) or (neighbor not in [i[1]for i in oheap]):
                came_from[neighbor] = current
                gscore[neighbor] = tentative_g_score
                fscore[neighbor] = tentative_g_score + heuristic(neighbor, goal,array)
                heapq.heappush(oheap, (fscore[neighbor], neighbor))
    return None


In [ ]:
import numpy as np
import random

# 定义路网矩阵
matrix = np.array([
    [2, 3, 5, 2, 3, 5, 2, 3],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [1, 1, 4, 1, 1, 4, 1, 1],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [1, 1, 4, 1, 1, 4, 1, 1],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [3, 2, 5, 3, 2, 5, 3, 2]
])

# 获取入口、出口和中途目的地的位置
entrances = np.argwhere(matrix == 2)
exits = np.argwhere(matrix == 3)
destinations = np.argwhere(matrix == 4)

# 选择随机入口、目的地和出口
random_entrance1 = tuple(random.choice(entrances))
random_destination1 = tuple(random.choice(destinations))
exit1 = find_nearest_exit(matrix,random_destination1)

# A* 算法查找从入口到目的地再到出口的最短路径
path_to_destination = Astar(matrix, random_entrance1, random_destination1)
path_to_exit = Astar(matrix, path_to_destination[-1], exit)
full_path1 = path_to_destination + path_to_exit[1:] + [exit]   # 合并两段路径，避免重复目的地点

# 选择随机入口、目的地和出口
random_entrance2 = tuple(random.choice(entrances))
random_destination2 = tuple(random.choice(destinations))
exit2 = find_nearest_exit(matrix,random_destination2)

# A* 算法查找从入口到目的地再到出口的最短路径
path_to_destination = Astar(matrix, random_entrance2, random_destination2)
path_to_exit = Astar(matrix, path_to_destination[-1], exit)
full_path2 = path_to_destination + path_to_exit[1:] + [exit]   # 合并两段路径，避免重复目的地点

In [ ]:
print(full_path1)

In [ ]:
import heapq
import numpy as np
import numpy as np
import random

# 定义任务类
class task:
    def __init__(self, entrance, destination, arrival_time,matrix):
        self.entrance = entrance
        self.destination = destination
        self.exit = self.find_nearest_exit(matrix,self.destination)
        self.path_to_destination = self.Astar(matrix, self.entrance, self.destination)
        self.path_to_exit = self.Astar(matrix, self.path_to_destination[-1], self.exit)
        self.full_path = self.path_to_destination + self.path_to_exit[1:] + [self.exit]
        self.arrival_time = arrival_time

    def heuristic(self, a, b, array,raw_orientation=None,new_orientation=None):
        """
        计算两个点之间的曼哈顿距离
        """
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def calculate_distance(self, pos1, pos2):
        return np.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

    #搜索map中最近的出口
    def find_nearest_exit(self, map_array, start):
        pos_of_exits = np.argwhere(map_array == 3)
        pos_of_exits.tolist()
        distances = [calculate_distance(start, (x, y)) for x, y in pos_of_exits]
        min_distance = min(distances)
        closest_position = pos_of_exits[distances.index(min_distance)]
        
        return (closest_position[0],closest_position[1])
    
    def find_neighbors(self, point,array=None):
        neighbors=[]
        if point[0] in [1,4,7]:
            neighbors.append((0,1))
        if point[0] in [2,5,8]:
            neighbors.append((0,-1))
        if point[1] in [0,3,6]:
            neighbors.append((1,0))
        if point[1] in [1,4,7]:
            neighbors.append((-1,0))
        return neighbors



    def Astar(self, array, start, goal):
        """
        A* 寻路算法
        """

        close_set = set()
        came_from = {}
        gscore = {start:0}
        fscore = {start:heuristic(start, goal, array)}
        oheap = []

        #初始化方向，位于上方入口（行索引为0），初始方向向下；位于下方入口，初始方向向上
        if start[0] == 0:
            last_orientation = (1,0)
        if start[0] == len(array)-1:
            last_orientation = (-1,0)
        if start[0] in [2,5]:
            last_orientation = (0,-1)
        if start[0] in [4,7]:
            last_orientation = (0,1)
        if start[1] in [1,4]:
            last_orientation = (-1,0)
        if start[1] in [3,6]:
            last_orientation = (1,0)
        
        heapq.heappush(oheap, (fscore[start], start))

        while oheap:
            # 标记一下这次有没有找到路
            current = heapq.heappop(oheap)[1]
            close_set.add(current)
            # 判断是否成功到达卸货单元
            for neighbor in [(0,1),(0,-1),(1,0),(-1,0)]:
                if (current[0]+neighbor[0],current[1]+neighbor[1]) == goal:
                    data = []
                    while current in came_from:
                        data.append(current)
                        current = came_from[current]
                    data.append(start)
                    return data[::-1]
            # 判断该点可达的邻居
            neighbors=find_neighbors(current,array)
            for i, j in neighbors:
                # 判断是否转向
                if current in came_from.keys():
                    if current[0] != came_from[current][0] or current[1] != came_from[current][1]:
                        last_orientation=(current[0]-came_from[current][0],current[1]-came_from[current][1]) #若停留的话last_orientation会成为(0,0),修改后停留时方向不变
                        
                neighbor = current[0] + i, current[1] + j

                # 约束：1.静态障碍物; 2.地图边界； 3.动态障碍
                if 0 <= neighbor[0] < array.shape[0]:
                    if 0 <= neighbor[1] < array.shape[1]:
                        # 判断是否为障碍物 此处可以改成其他限制条件
                        if array[neighbor[0]][neighbor[1]] == 5:
                            continue
                        if array[neighbor[0]][neighbor[1]] == 4 and neighbor != goal:
                            continue
                    else:
                        # 超出边界
                        continue
                else:
                    # 超出边界
                    continue
                
                # 如果要转向，距离额外加1
                if last_orientation!=(i,j):
                    tentative_g_score = gscore[current] + heuristic(current, neighbor, array) + 1  
                else:
                    tentative_g_score = gscore[current] + heuristic(current, neighbor, array)
                # 如果距离更远，排除
                if neighbor in close_set  and tentative_g_score > gscore.get(neighbor, 0):
                    continue
                # 如果距离更近，更新
                if  tentative_g_score <= gscore.get(neighbor, 0) or (neighbor not in [i[1]for i in oheap]):
                    came_from[neighbor] = current
                    gscore[neighbor] = tentative_g_score
                    fscore[neighbor] = tentative_g_score + heuristic(neighbor, goal,array)
                    heapq.heappush(oheap, (fscore[neighbor], neighbor))
        return None


# 定义路网矩阵
matrix = np.array([
    [2, 3, 5, 2, 3, 5, 2, 3],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [1, 1, 4, 1, 1, 4, 1, 1],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [1, 1, 4, 1, 1, 4, 1, 1],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 0, 0],
    [3, 2, 5, 3, 2, 5, 3, 2]
])

entrances = np.argwhere(matrix == 2)
exits = np.argwhere(matrix == 3)
destinations = np.argwhere(matrix == 4)

# 生成随机到达任务时间列表
arrival_time_list = [random.randint(0,20) for i in range(10)]
arrival_time_list.sort()



# 生成任务列表
task_list = []
for i in range(len(arrival_time_list)):
    task_list.append(task(tuple(random.choice(entrances)),tuple(random.choice(destinations)),arrival_time_list[i],matrix))
    # 若任务时间和起点和之前的任务相同，则重置该任务
    if i>0:
        while task_list[i].entrance == task_list[i-1].entrance and task_list[i].arrival_time == task_list[i-1].arrival_time:
            task_list[i] = task(tuple(random.choice(entrances)),tuple(random.choice(destinations)),arrival_time_list[i],matrix)


for i in range(len(task_list)):
    print(f"任务{i}开始时间:",task_list[i].arrival_time,f"任务{i}路径:",task_list[i].full_path)


In [103]:
# 定义车辆类
class AGV:
    def __init__(self, position, speed=0, needs_turn=False, path=None, state=1,direction=None):
        self.position = position
        self.speed = speed
        self.needs_turn = needs_turn
        self.path = path[1:]
        self.state = state # 1:负载 0:空载
        self.direction = direction # (1,0)向下 (-1,0)向上 (0,1)向右 (0,-1)向左

        # 初始化方向
        if position[0] in [0,9] and position[1] in [0,3,6]:
            self.direction = (1,0)
        if position[0] in [0,9] and position[1] in [1,4,7]:
            self.direction = (-1,0)

    def update_speed(self, matrix):
        # 加速规则
        self.speed = min(V_max, self.speed + 1)

        # 减速规则,防止碰撞
        dn = self.calculate_dn(matrix)
        self.speed = min(self.speed, dn)


    def calculate_dn(self, matrix):
        # 计算前方空闲元胞数量
        x, y = self.position
        dn = 0
        if self.direction == (1,0):
            for i in range(x+1, matrix.shape[0]):
                if matrix[i][y] != -1:
                    dn += 1
                else:
                    break
        if self.direction == (-1,0):
            for i in range(x-1, -1, -1):
                if matrix[i][y] != -1:
                    dn += 1
                else:
                    break
        if self.direction == (0,1):
            for i in range(y+1, matrix.shape[1]):
                if matrix[x][i] != -1:
                    dn += 1
                else:
                    break
        if self.direction == (0,-1):
            for i in range(y-1, -1, -1):
                if matrix[x][i] != -1:
                    dn += 1
                else:
                    break
        #print("前方空闲元胞数量:",dn)
        return dn

    def update_position(self, matrix):
        # 更新车辆位置
        if not self.needs_turn:
            if len(self.path) > 0:
                matrix[self.position[0]][self.position[1]] = 0
                self.position = ((self.position[0] + self.speed * self.direction[0]), (self.position[1] + self.speed * self.direction[1]))
                matrix[self.position[0]][self.position[1]] = -1
                self.path = self.path[1:]
            else:
                print("AGV已到达目的地")
                matrix[self.position[0]][self.position[1]] = 0
        # 需要转弯，原地停留
        else:
            self.position = self.position
            self.direction = ((self.path[0][0] - self.position[0]),(self.path[0][1] - self.position[1]))
            self.needs_turn = False
            

    def check_turn(self):
        # 检查是否需要转向（根据实际情况实现）,agv到达终点时不需要转向
        if len(self.path) > 0:
            if ((self.path[0][0] - self.position[0]),(self.path[0][1] - self.position[1])) != self.direction:
                self.needs_turn = True
        else:
            self.needs_turn = False
        

# 初始化车辆
agvs = [AGV(full_path1[0], 0, False, full_path1, 1), AGV(full_path2[0], 0, False, full_path2, 1)]

# 模拟单步
def simulate_step(matrix, agvs):
    for agv in agvs:
        agv.update_speed(matrix)
        agv.check_turn()
        agv.update_position(matrix)

    # 这里可以添加代码以更新路网矩阵
    return matrix, agvs

'''# 定义最大速度
V_max = 1
# 定义10*8的路网矩阵
matrix = np.full((10,8), 0)

# 时间步为0初始化AGV信息
for j in range(len(agvs)):
    matrix[agvs[j].position[0]][agvs[j].position[1]] = -1
    #print(f"AGV{j}当前位置：",agvs[j].position,f"AGV{j}当前速度：",agvs[j].speed,f"AGV{j}路径：",agvs[j].path)
print("第",0,"步")
print(matrix)
   

# 运行模拟
time_Step = 12
for i in range(time_Step):
    print("第",i+1,"步")
    matrix, agvs = simulate_step(matrix, agvs)
    for j in range(len(agvs)):
        #输出AGV1的信息      
        print(f"AGV{j}当前位置：",agvs[j].position,f"AGV{j}当前速度：",agvs[j].speed,f"AGV{j}路径：",agvs[j].path)
    print(matrix)'''

'# 定义最大速度\nV_max = 1\n# 定义10*8的路网矩阵\nmatrix = np.full((10,8), 0)\n\n# 时间步为0初始化AGV信息\nfor j in range(len(agvs)):\n    matrix[agvs[j].position[0]][agvs[j].position[1]] = -1\n    #print(f"AGV{j}当前位置：",agvs[j].position,f"AGV{j}当前速度：",agvs[j].speed,f"AGV{j}路径：",agvs[j].path)\nprint("第",0,"步")\nprint(matrix)\n   \n\n# 运行模拟\ntime_Step = 12\nfor i in range(time_Step):\n    print("第",i+1,"步")\n    matrix, agvs = simulate_step(matrix, agvs)\n    for j in range(len(agvs)):\n        #输出AGV1的信息      \n        print(f"AGV{j}当前位置：",agvs[j].position,f"AGV{j}当前速度：",agvs[j].speed,f"AGV{j}路径：",agvs[j].path)\n    print(matrix)'

In [99]:
arrival_time_list

[2, 3, 5, 5, 6, 6, 7, 11, 12, 13]

In [123]:
import copy
import matplotlib.pyplot as plt
import time

agvs = []
agv_matrix = np.full((10,8), 0)
time_Step = 30
tmp = copy.deepcopy(arrival_time_list) #拷贝到达时间列表
for i in range(time_Step):
    print("第",i,"步")
    agv_matrix, agvs = simulate_step(agv_matrix, agvs)
    for j in tmp:
        if j == i:
            print(f"第{j}时间步有新任务到达")
            # 初始化车辆
            agvs.append(AGV(task_list[arrival_time_list.index(j)].full_path[0], 0, False, task_list[arrival_time_list.index(j)].full_path, 1))
            tmp.remove(j)
            for k in range(len(agvs)):
                agv_matrix[agvs[k].position[0]][agvs[k].position[1]] = -1
    print(agv_matrix)

    # 可视化agv_matrix
    plt.figure(figsize=(10,8))
    plt.imshow(agv_matrix)
    plt.show()

    time.sleep(2)


第 0 步
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


<IPython.core.display.Javascript object>

Exception ignored in: <function Comm.__del__ at 0x0000010DC9E03620>
Traceback (most recent call last):
  File "d:\Anaconda\lib\site-packages\ipykernel\comm\comm.py", line 76, in __del__
    self.close()
  File "d:\Anaconda\lib\site-packages\ipykernel\comm\comm.py", line 116, in close
    self.kernel.comm_manager.unregister_comm(self)
  File "d:\Anaconda\lib\site-packages\ipykernel\comm\manager.py", line 56, in unregister_comm
    comm = self.comms.pop(comm.comm_id)
KeyError: ('957eb3a96b194461bcc19800d9c1d6f8',)
Exception ignored in: <function Comm.__del__ at 0x0000010DC9E03620>
Traceback (most recent call last):
  File "d:\Anaconda\lib\site-packages\ipykernel\comm\comm.py", line 76, in __del__
    self.close()
  File "d:\Anaconda\lib\site-packages\ipykernel\comm\comm.py", line 116, in close
    self.kernel.comm_manager.unregister_comm(self)
  File "d:\Anaconda\lib\site-packages\ipykernel\comm\manager.py", line 56, in unregister_comm
    comm = self.comms.pop(comm.comm_id)
KeyError: ('

第 1 步
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


<IPython.core.display.Javascript object>

第 2 步
第2时间步有新任务到达
[[-1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]]


<IPython.core.display.Javascript object>

第 3 步
第3时间步有新任务到达
[[ 0  0  0  0  0  0  0  0]
 [-1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 -1]]


<IPython.core.display.Javascript object>

KeyboardInterrupt: 

第4步


<IPython.core.display.Javascript object>

KeyboardInterrupt: 